In [1]:
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
import requests

import random
import time
import pandas as pd
import math
from urllib import parse
import os

In [2]:
my_headers_Get = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

def getPage(url):
    delay = 0.1*random.random()
    time.sleep(delay)
    s = requests.Session()
    return s.get(url, headers=my_headers_Get)

def robust_select(soup,SELECTOR):
    if type(SELECTOR) is not str:
        return None
    else:
        res = soup.select(SELECTOR)
        if res is None:
            return None
        else:
            if len(res) == 1:
                return res[0].text
            else:
                rescombined = ''
                for part in res:
                    rescombined += part.text
                return rescombined

In [24]:
city = '上海'

websiteInfo = pd.read_csv('settings/city_govs.csv',index_col='city')

start_urls = websiteInfo['url'][city]
MAIN_PAGE_ENCODING = websiteInfo['main_page_encoding'][city]
SUB_PAGE_ENCODING = websiteInfo['sub_page_encoding'][city]

#Load the selectors for the city
selectorInfo = pd.read_csv('settings/selector_settings.csv',index_col='city')
NEWS_SELECTOR = selectorInfo['news_selector'][city]
URL_SELECTOR = selectorInfo['url_selector'][city]
NEXT_PAGE_SELECTOR = selectorInfo['next_page_selector'][city]
TITLE_SELECTOR = selectorInfo['title_selector'][city]
DATE_SELECTOR = selectorInfo['date_selector'][city]
SOURCE_SELECTOR = selectorInfo['source_selector'][city]
DPRT_SELECTOR = selectorInfo['department_selector'][city]
CONTENT_SELECTOR = selectorInfo['content_selector'][city]

#Test url
test_url = 'http://www.shanghai.gov.cn/nw2/nw2314/nw2319/nw12344/u26aw62581.html'

page = getPage(test_url)
soup = BeautifulSoup(page.content.decode(MAIN_PAGE_ENCODING,'ignore'), 'html.parser')
#options = webdriver.ChromeOptions()
#options.add_argument('headless')

#driver = webdriver.Chrome(options=options)
#driver.get(start_urls)
#soup = BeautifulSoup(driver.page_source, 'html.parser')

In [25]:
soup.select(CONTENT_SELECTOR)

[<p>
 	各区卫生健康委，申康医院发展中心、有关大学、中福会，各专科医师规范化培训医院，上海市医师协会，上海市卫生人才交流服务中心，上海市住院医师规范化培训事务中心:</p>,
 <p>
 	　　根据本市各级医疗卫生机构临床岗位需求和各专科医师规范化培训医院培训能力情况，市卫生健康委制定了《2019年上海市专科医师规范化培训招录计划》（见附件），计划招录总人数为1400名。</p>,
 <p>
 	　　请有关部门根据职责分工，做好专科医师招录工作的指导和管理。承担培训任务医院的各职能部门要通力协作，认真做好专科医师规范化培训招录工作。培训基地所在科室录用的培训对象须全部纳入专科医师规范化培训。</p>,
 <p>
 	　　特此通知。</p>,
 <p>
 	　　附件：2019年上海市专科医师规范化培训招录计划</p>,
 <p style="text-align: right;">
 	　　上海市卫生健康委员会</p>,
 <p style="text-align: right;">
 	　　2019年8月28日</p>]

In [7]:
page = getPage(test_url)
soup = BeautifulSoup(page.content.decode(MAIN_PAGE_ENCODING,'ignore'), 'html.parser')

In [22]:
CONTENT_SELECTOR = 'div#ivs_content>p'
title = robust_select(soup,TITLE_SELECTOR)
content = robust_select(soup,CONTENT_SELECTOR)
date = robust_select(soup,DATE_SELECTOR)

In [19]:
content

'\n\t各区卫生健康委，申康医院发展中心、有关大学、中福会，各专科医师规范化培训医院，上海市医师协会，上海市卫生人才交流服务中心，上海市住院医师规范化培训事务中心:\n\t\u3000\u3000根据本市各级医疗卫生机构临床岗位需求和各专科医师规范化培训医院培训能力情况，市卫生健康委制定了《2019年上海市专科医师规范化培训招录计划》（见附件），计划招录总人数为1400名。\n\t\u3000\u3000请有关部门根据职责分工，做好专科医师招录工作的指导和管理。承担培训任务医院的各职能部门要通力协作，认真做好专科医师规范化培训招录工作。培训基地所在科室录用的培训对象须全部纳入专科医师规范化培训。\n\t\u3000\u3000特此通知。\n\t\u3000\u3000附件：2019年上海市专科医师规范化培训招录计划\n\t\u3000\u3000上海市卫生健康委员会\n\t\u3000\u30002019年8月28日'

In [9]:
robust_select(soup,TITLE_SELECTOR)

'上海市卫生健康委员会关于公布2019年上海市专科医师规范化培训招录计划的通知\n\t\t\t\t\t\t \n\t\t\t\t\t\t \n( 2019年09月06日 )\n\n'

In [10]:
TITLE_SELECTOR

'div#ivs_title'